**Welcome everyone**

In this notebook I will do some analysis to understand the data and get a good idea about the features
that I will use to train a model that predict the Heart Disease. 

**I will pretend to be machine learning model**

First of all I'm not a doctor, my major is industrial engineering and I have no idea about these features that 
I'm going to use so I will pretend to be machine learning model thak looks to the data as numbers 
find a good representations that will help it learn and trying to find a mapping between the the x and y. 

also I will try to understand the Heart Disease from the data as a ML model, let's see if I can :) also I will try 
to measure my perfomance on the test set it will be interesting thing to try because my background is industrial
engineering not medicine. 

Let us start with some analysis to understand the data. 

**Contents:**
    
1- Understand the data. <br>
2- Data preperation and visualization using PCA. <br>
3- Classification models & Kfold cross validation. <br>
4-Deep learning input pipeline and classification model.<br>
5- Final Analysis and comparison.<br>
6- Conclusion.<br>
7- Thank you.<br>

# 1- Understand the data 

In [ ]:
from pathlib import Path
import os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
from scipy import stats
#-------------------------------------------------------------------------
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
#-------------------------------------------------------------------------
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
#-------------------------------------------------------------------------
from sklearn.decomposition import PCA
#-------------------------------------------------------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
#-------------------------------------------------------------------------
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.metrics import classification_report

In [ ]:
data_path =Path(r"../input/heart-disease-uci/heart.csv")
data = pd.read_csv(data_path)

Features:
    
- age <br>
- sex <br>
- chest pain type (4 values) <br>
- resting blood pressure <br>
- serum cholestoral in mg/dl <br>
- fasting blood sugar > 120 mg/dl <br>
- resting electrocardiographic results (values 0,1,2) <br>
- maximum heart rate achieved <br>
- exercise induced angina <br>
- oldpeak = ST depression induced by exercise relative to rest <br>
- the slope of the peak exercise ST segment <br>
- number of major vessels (0-3) colored by flourosopy <br>
- thal: 3 = normal; 6 = fixed defect; 7 = reversable defect <br>

The number of training datais very small so it will be challenge. I will use Kfold cross validation. 

## 1.1- First look

In [ ]:
data.head()

## 1.2- Check for missing values

There are no missing values in the data so we cak continue 

In [ ]:
data.isnull().sum()

## 1.3- Check for missing values

In [ ]:
all_features = ['cp','restecg', 'slope',
                'ca', 'thal',
                'age', 'trestbps',
                'chol', 'thalach', 'oldpeak']

plt.figure(figsize=(20,15))

for ind, feat in enumerate(all_features):
    
    plt.subplot(4,4,ind+1)
    sns.boxplot(y=feat,data=data)

Features ('cp', 'thal', 'trestbps', 'chol', 'thalach', 'oldpeak') indicate some outliers 
I will deal with that in the data preprocessing section.

## 1.4- data types

In [ ]:
data.info()

In [ ]:
categorical_featurs = ['sex', 'cp', 'fbs','restecg', 'exang', 'slope', 'ca', 'thal']

numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

## 1.5- utility functions

In [ ]:
# utilities

def plot_histogram(column):
    have_the_disease = data[data['target']==1][column].value_counts()
    no_disease  = data[data['target']==0][column].value_counts()
    df = pd.DataFrame([have_the_disease, no_disease])
    df.index = ['Have the disease','No disease']
    df.plot(kind='bar',stacked=True, title=str(column))
    
    
def categorical_data_analysis(feature):
    
    print(data[feature].value_counts())
    print("---------------------------------")

    print("{} - have the disease".format(feature))
    print(data[data['target']==1][feature].value_counts())
    print('--------------------------------------')


    print("{} - do not have the disease".format(feature))
    print(data[data['target']==0][feature].value_counts())
    print('--------------------------------------')

    plot_histogram(feature)
    
def numerical_data_analysis(feature): 
    
    print("- Statistics of the {} feature for the people  have the disease".format(feature))
    print(data[data['target']==1][feature].describe())
    print('-------------------------------------------')

    print("- Statistics of the trestbps feature for the people do not have the disease")
    print(data[data['target']==0][feature].describe())
    print('-------------------------------------------')


    plt.hist(data[data['target']==1][feature],
            color='red',
            edgecolor='black',
            alpha=0.4,
            label='{} - have the disease'.format(feature))

    plt.hist(data[data['target']==0][feature],
            color='blue',
            edgecolor='black',
            alpha=0.4,
            label='{} - dont have the disease'.format(feature))

    plt.grid(True)
    plt.legend(loc='upper right')
    

## 1.6- take alook at the features one by one

### 1.6.1- Age

In [ ]:
numerical_data_analysis('age')

As we can see from the statistics of the ages for the people have and dont have the disease
that the mean and the standard deviation are so close for both

and from the means and the histogram we can say that the younger people are more susceptible to disease !!!

And due to the little difference between them I can say that the ML model will not will not benefit much from 
this feature so I will use **(binning)** to try to get a useful info for the model. 

### 1.6.2- sex

In [ ]:
categorical_data_analysis('sex')

As we can see:
    
- The number of males in this dataset is more than twice the number of females, 
- The number of males that have the disease is higher than females that have the disease
- The number of males that do not have the disease is higher than females that have the disease

So we may encounter some **bayes problems**. 

### 1.6.3- cp - chest pain type

In [ ]:
data['cp'].value_counts()
print("---------------------------------")

plot_histogram('cp')

 I can not identify the types in name, and what the dataset creator is meant by chest pain type. 

but as I said previously I want to **think like a ML model** and identify patterns from row data, so from the above figure 
I can conclude that the type 0 means no pain science the majority of the people have no disease are having the type 0 pain
and some of the people have the disease do not have does not feel any type of pain. 

And the most frequent pain type between people have the disease is type 2. 

### 1.6.4- trestbps - resting blood pressure  

Blood pressure (BP) is the pressure of circulating blood against the walls of blood vessels.
Most of this pressure results from the heart pumping blood through the circulatory system.
When used without qualification, the term "blood pressure" refers to the pressure in the large arteries.
Blood pressure is usually expressed in terms of the systolic pressure (maximum pressure during one heartbeat) 
over diastolic pressure (minimum pressure between two heartbeats) in the cardiac cycle. It is measured in millimeters
of mercury (mmHg) above the surrounding atmospheric pressure.

**Source:** <a href="https://en.wikipedia.org/wiki/Blood_pressure">wikipedia-Blood pressure/<a>

In [ ]:
numerical_data_analysis('trestbps')

as we can see from  the figure and the discribtive statistics the people have the disease tend to have
a lower resting blood pressure. 

### 1.6.5- chol - serum cholestoral in mg/dl   

In [ ]:
numerical_data_analysis('chol')

As we can see the people have the disease tend to have a lower serum cholesterol in mg/dl
the figure may confuse you but the x axis is chol and the y axis count the number of people with that chol. 
also we notice an outlier in the people have the disease that will negatively affect the training so 
in the data preperation section I will get red of that. 

In [ ]:
#The outlayer

chol_1 = data[data['target']==1]['chol'] 
chol_1[chol_1 > 500]

You can read more about serum cholesterol at <a href="https://www.healthline.com/health/serum-cholesterol">healthline- What Is Serum Cholesterol and Why Is It Important?/<a>


### 1.6.6- fbs - fasting blood sugar > 120 mg/dl 

In [ ]:
categorical_data_analysis('fbs')

Firs of all the number of people with  fasting blood sugar > 120 mg/dl (class 1) is very small compared to number of people
fasting blood sugar < 120 mg/dl (class 0)

second the numbers are so closed to each others, where the number of people that have the disease and fasting blood sugar > 120 mg/dl 
(class 1) is almost equal to the number of people that do not have the disease and fasting blood sugar < 120 mg/dl 
(class 0). 


So I think that this feature will be useless to the training model, this feature will not help the model 
to classify a person to target 1 or 0. 

I will use the Feature Selection functions in sklearn to see if my hypothesis is right or not. 

### 1.6.7- restecg - resting electrocardiographic results (values 0,1,2)

In [ ]:
categorical_data_analysis('restecg')

unfortunately the number of observations for restecg (class2) is very small it's just 4 observations :(
the model will not generalize well in this class, while the numbers in the other two classes is balanced and that's good. 

### 1.6.8- thalach - maximum heart rate achieved 

In [ ]:
numerical_data_analysis('thalach')

people have the disease have higher mean maximum heart rate achieved of 158.47 also they have a higher min and max 
of (96, 202). 

from the figure above the distribution of the (thalach - dont have the disease) have a higer deiviation than the 
distribution of the (thalach - have the disease) where most of the obsevations concentrated around  the mean. 

The two distribution are difeerent in parameters so it will help the calssification model during training. 

### 1.6.9- exang - exercise induced angina  

In [ ]:
categorical_data_analysis('exang')

### 1.6.10- oldpeak - oldpeak = ST depression induced by exercise relative to rest  

In [ ]:
 numerical_data_analysis('oldpeak')

### 1.6.11- slope - the slope of the peak exercise ST segment 

In [ ]:
categorical_data_analysis('slope')

### 1.6.12- ca - number of major vessels (0-3) colored by flourosopy 

In [ ]:
categorical_data_analysis('ca')

### 1.6.13- thal - thal: 3 = normal; 6 = fixed defect; 7 = reversable defect  

In [ ]:
categorical_data_analysis('thal')

The last 5 features I will talk about them briefly because they need an expert in medical sciences.


In general for the last 3 categorical features the number of observations is imabalnced, such as 
(thal) feature where there are just 4 observations of the (class 0) we can remove them from the dataset
before training because they will affect the training negatively. And for the number of major vessels (ca) feature, most of the people have the disease have number of major vessels 0 (class 0)
this will be helpful during the training process. 


For the (the slope of the peak exercise ST segment) feature most of the people that have the disease have the (class 2) of this feature 
while most of the people dont have the disease have the (class 1) of this feature. This also will be helpful during training. 

## 1.7- correlations

correlation: is a number between –1.0 and +1.0. positive corelation meaning both variables move in the same direction
if we increase one of them the other one also increase and if we decrease one of them the other one will increase. While negative correlations means that the both variables are moveing in opposite  direction. <br>

- Positive Correlation: both variables change in the same direction.<br>
- Neutral Correlation: No relationship in the change of the variables.<br>
- Negative Correlation: variables change in opposite directions.<br>
    
It is important to note that correlation doesn't imply causation. 
Correlation quantifies the strength of the relationship between the features of a dataset. <br>


**Source:** <a href="https://re-thought.com/exploring-correlation-in-python/">Exploring Correlation in Python: Pandas, SciPy/<a>

There are several statistics that you can use to quantify correlation, three of them: <br>

- Pearson’s r. Can be used to summarize the strength of the linear relationship between two data samples.
To use Pearson both valriables must be numenrical and normally distributed. Pearson’s correlation coefficient
calculated as the covariance of  the two variables divided by the product of the standard deviation of each data sample.<br>

- Spearman’s ρ (rho): does not assume that data is from a specific distribution, also does not require the variables to be normally distributed nor the relationship between them to be linear, and does not  require the variables to be numerical,
Spearman’s rank correlation tests whether two ordinal or/and quantitative variables are dependent.<br>

Spearman's correlation coefficient, (ρ, also signified by rs) measures the strength and direction of the association between
two ranked variables. this coefficient is between -1 and +1 The higher the magnitude of ρ (in the positive or negative directions),
the stronger the relationship.<br>


- Kendall’s τ (tau) <br>

Correlation can be useful in data analysis and modelling to better understand the relationships between variables. 
So it is important that you have a good understanding of it before you attempt a data analysis or modelling.<br>


for more information visit: <a href="https://re-thought.com/exploring-correlation-in-python/">Exploring Correlation in Python: Pandas, SciPy/<a>

In [ ]:
#I will use here Spearman’s 

In [ ]:
categorical_featurs = ['sex', 'cp', 'fbs','restecg', 'exang', 'slope', 'ca', 'thal']

numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

features = ['age', 'cp', 'chol', 'restecg', 'thalach', 'oldpeak', 'slope', 'ca', 'thal', 'target']

corr = data[features].corr(method ='spearman')
corr.style.background_gradient(cmap='coolwarm')

as we can see all the variables are related with the target variable either positively negatively.


If you notice the analysis in the previos section I taked about each valriable if it will have 
a large or small contribution in the training process based on charts in the previous sections. 


In the previous section I said that the variables (thal, ca, slope, thalach, cp) will have a good contribution
during the training process and as we can see they have the highest Spearman's correlation coefficient with the target 
value. All these are assumptions we will se if that true during the feature selection and training processes.


# 2- Data preperation and visualization using PCA.

## 2.1- data cleaning 

The data does not contain any missing values so in this section I'll get rid of outliers

The following code will: 
    
- For each column, it first computes the Z-score of each value in the column, relative to the column mean and standard deviation.

- It then takes the absolute Z-score because the direction does not matter, only if it is below the threshold.

- all(axis=1) ensures that for each row, all column satisfy the constraint.

- Finally, the result of this condition is used to index the dataframe.


**Source:** <a href="https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame">Detect and exclude outliers in Pandas data frame/<a>

In [ ]:
data_cleaned = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]

In [ ]:
all_features = ['cp','restecg', 'slope',
                'ca', 'thal',
                'age', 'trestbps',
                'chol', 'thalach', 'oldpeak']

plt.figure(figsize=(20,15))

for ind, feat in enumerate(all_features):
    
    plt.subplot(4,4,ind+1)
    sns.boxplot(y=feat,data=data_cleaned)

we now have less outliers as we can see from the boxplots, I will not remove the remained outliers due to the small amount 
of data we have. 

## 2.2- Feature engineering 

In this section data will be prepared by transfirming the row data we have to useful representation can be 
used by the models to perform the classification. 

- I will use **Discretization** for the age feature. <br>
- **One-hot encoding** for the categorical features. <br>
- And for the numerizal featurs: **extract polynoial features**, **feature selection** and then **normalization** <br>

### 2.2.1- Discretization

In [ ]:
ages = set(np.array(data_cleaned['age']))
print(ages)

Here we will partition the input range into fixed number of bins, then a data point will be represented
by which bin it fails into.

As we can see the ages is between 29 and 77 I will create three bins the first one include the ages between 20 and 39, 
the second one contain ages between 40 and 59, the third one contain ages between 60 and 79. 

In [ ]:
bins = [25, 40, 60, 79]

# now we record for each data points which bin it fails into
which_bin = np.digitize(data_cleaned['age'], bins=bins)

In [ ]:
which_bin

In [ ]:
binned_ages = pd.Series(which_bin)

In [ ]:
# removing the pld age column
ready_for_preprocessing_data = data_cleaned.drop('age', axis=1)

In [ ]:
# adding the new age column
ready_for_preprocessing_data = ready_for_preprocessing_data.assign(age=binned_ages.values)

And by now the ages are ready for one_hot encoding 

### 2.2.2- Preprocessing input pipeline 

In [ ]:
categorical_featurs = ['sex', 'cp', 'fbs','restecg', 'exang', 'slope', 'ca', 'thal', 'age']
numerical_features = ['trestbps', 'chol', 'thalach', 'oldpeak']

In [ ]:
X = ready_for_preprocessing_data.drop('target', axis=1)
y = ready_for_preprocessing_data['target']

In [ ]:
# pipeline for numerical data as discribed before
numerica_pipline = Pipeline([
   ("poly", PolynomialFeatures(degree=2, include_bias=False)),
   ("norm", MinMaxScaler()),
   ("selection", RFE(LogisticRegression(max_iter=10000), n_features_to_select=4))
])


preprocessing_pipeline = ColumnTransformer([
    ('num', numerica_pipline, numerical_features),
    ('cat', OneHotEncoder(), categorical_featurs)
])

X_prepered = preprocessing_pipeline.fit_transform(X, y)

In [ ]:
X_prepered.shape

## 2.3- visualization after compressing the data using PCA

Some of the machine learning problems involve a dataset with a very high dimensionality,
which will extremely slowdown the learning process, and make the learning harder for the algorithm this problem 
is called Curse of dimensionality. Dimensionality reduction one of the techniques that is widely used to solve the
problem but by reducing the dimensionality, we will lose some of the information that contained the dataset,
the benefit will be to speed up the learning process.

Principle Component Analysis aims to reducing the dimensionality of the data while preserving the largest amount
of information (variance) presented in the dataset.

you can learn more about Principle Component Analysis (PCA) here: <a href="https://www.kaggle.com/general/253753">Principle Component Analysis (PCA) fully Explained!!/<a>

In [ ]:
pca_3 = PCA(n_components=3)
pca_3.fit(X_prepered)
X_pca_3 = pca_3.transform(X_prepered)

In [ ]:
pca_df_3 = pd.DataFrame(X_pca_3, columns={'feature1', 'feature2', 'feature3'})
pca_df_3 = pca_df_3.assign(target=y.values)

In [ ]:
class1_features = pca_df_3[pca_df_3['target']==1]
class0_features = pca_df_3[pca_df_3['target']==0]

In [ ]:
x_class1 = class1_features['feature1']
y_class1 = class1_features['feature2']
z_class1 = class1_features['feature2']

x_class0 = class0_features['feature1']
y_class0 = class0_features['feature2']
z_class0 = class0_features['feature2']


fig = plt.figure(figsize=(30, 30))


ax = fig.add_subplot(231, projection = '3d')
ax.scatter(x_class1, y_class1, z_class1, 'or', s=150, label='Class_1')
ax.scatter(x_class0, y_class0, z_class0, 'ob', s=150, label='Class_0')
ax.view_init(0, 90)
plt.legend()

ax = fig.add_subplot(232, projection = '3d')
ax.scatter(x_class1, y_class1, z_class1, 'or', s=150, label='Class_1')
ax.scatter(x_class0, y_class0, z_class0, 'ob', s=150, label='Class_0')
ax.view_init(0, 180)
plt.legend()

ax = fig.add_subplot(233, projection = '3d')
ax.scatter(x_class1, y_class1, z_class1, 'or', s=150, label='Class_1')
ax.scatter(x_class0, y_class0, z_class0, 'ob', s=150, label='Class_0')
ax.view_init(270, 270)
plt.legend()

In [ ]:
pca_2 = PCA(n_components=2)
pca_2.fit(X_prepered)
X_pca_2 = pca_2.transform(X_prepered)

pca_df_2 = pd.DataFrame(X_pca_2, columns={'feature1', 'feature2'})
pca_df_2 = pca_df_2.assign(target=y.values)

class1_features_2 = pca_df_2[pca_df_3['target']==1]
class0_features_2 = pca_df_2[pca_df_3['target']==0]

plt.plot(class1_features_2['feature1'], class1_features_2['feature2'], 'or', label='Class_1')
plt.plot(class0_features_2['feature1'], class0_features_2['feature2'], 'ob', label='Class_0')

plt.grid(True)
plt.legend()

As we can see from the figures above the data is not linearly separable. This will help us alot to choose the right 
algorithms.

## 2.4- Splitting the data

The data will be splitted to train and test sets and the test set will be used for the final model testing.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_prepered,
                                                    y,
                                                    test_size=0.15,
                                                    shuffle=True,
                                                    random_state=0)

# 3- Classification models & Kfold cross validation.

The following Machine Learning algorithms will be tested: 
    
    1- Logistic Regression
    2- DecisionTree Classifier
    3- Ensemble Methods:
        a- Random Forest
        b- Voting Classifiers
        c- bagging and Pasting
        d- Boosting

## 3.1- Logistic Regression

In [ ]:
lr_model = LogisticRegression(max_iter=1000, random_state=0)
lr_model.fit(x_train , y_train)
lr_train_score = lr_model.score(x_train , y_train)

print("Logistic Regression Training Score: {:.3F}".format(lr_train_score))


# LogisticRegression evaluated using shuffle-split cross-validation 
lr_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.15, n_splits=5, random_state=0)
lr_val_scores = cross_val_score(lr_model, x_train , y_train, cv=lr_shuffle_split)
print("Logistic Regression Cross validation Score: {:.3F}".format(np.mean(lr_val_scores)))

## 3.2- DecisionTree Classifier

In [ ]:
# overfitting

dt_model = DecisionTreeClassifier(random_state=0)
dt_model.fit(x_train , y_train)
dt_train_score = dt_model.score(x_train , y_train)

print("DecisionTree Classifier Training Score: {:.3F}".format(dt_train_score))


# DecisionTreeClassifier evaluated using shuffle-split cross-validation 
dt_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.2, n_splits=5, random_state=0)
dt_val_scores = cross_val_score(dt_model, x_train , y_train, cv=dt_shuffle_split)
print("DecisionTree Classifier Cross validation Score: {:.3F}".format(np.mean(dt_val_scores)))

## 3.3- Ensemble Methods

### 3.3.1- Random Forest

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,
                                 max_leaf_nodes=10,
                                 bootstrap=False,
                                 max_samples=100,
                                 n_jobs=-1,
                                 random_state=0)

rf_model.fit(x_train , y_train)
rf_train_score = rf_model.score(x_train , y_train)

print("Random Forest Training Score: {:.3F}".format(rf_train_score))


# RandomForestClassifier evaluated using shuffle-split cross-validation 
rf_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.2, n_splits=5, random_state=0)
rf_val_scores = cross_val_score(rf_model, x_train , y_train, cv=rf_shuffle_split)
print("Random Forest Cross validation Score: {:.3F}".format(np.mean(rf_val_scores)))

### 3.3.2- Voting Classifiers

In [ ]:
# Hard Voting 
v_log_clf = LogisticRegression(max_iter=1000, random_state=0)
v_random_forest = RandomForestClassifier()
v_dt = DecisionTreeClassifier(random_state=0)


hard_voting_model = VotingClassifier(estimators=[("lr",v_log_clf),
                                            ("rf",v_random_forest),
                                            ("DT",v_dt)],
                               voting='hard')

hard_voting_model.fit(x_train , y_train)
hard_voting_train_score = hard_voting_model.score(x_train , y_train)

print("Hard Voting Classifiers Training Score: {:.3F}".format(hard_voting_train_score))


# VotingClassifier evaluated using shuffle-split cross-validation 
hard_voting_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.2, n_splits=5, random_state=0)
hard_voting_val_scores = cross_val_score(hard_voting_model, x_train , y_train, cv=hard_voting_shuffle_split)
print("Hard Voting Classifiers Cross validation Score: {:.3F}".format(np.mean(hard_voting_val_scores)))

In [ ]:
# Soft Voting
v_log_clf = LogisticRegression(max_iter=1000, random_state=0)
v_random_forest = RandomForestClassifier()
v_dt = DecisionTreeClassifier(random_state=0)


soft_voting_model = VotingClassifier(estimators=[("lr",v_log_clf),
                                            ("rf",v_random_forest),
                                            ("DT",v_dt)],
                               voting='soft')

soft_voting_model.fit(x_train , y_train)
soft_voting_train_score = soft_voting_model.score(x_train , y_train)

print("Soft Voting Classifiers Training Score: {:.3F}".format(soft_voting_train_score))


# VotingClassifier evaluated using shuffle-split cross-validation 
soft_voting_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.2, n_splits=5, random_state=0)
soft_voting_val_scores = cross_val_score(soft_voting_model, x_train , y_train, cv=soft_voting_shuffle_split)
print("Soft Voting Classifiers Cross validation Score: {:.3F}".format(np.mean(soft_voting_val_scores)))

### 3.3.3- bagging and Pasting

#### 3.3.3.1- bagging

In [ ]:
rf_bagging_model = BaggingClassifier(base_estimator=RandomForestClassifier(),
                                     n_estimators=100,
                                     bootstrap=True, #The difference between baggong and pasting
                                     max_samples=100,
                                     n_jobs=-1,
                                     random_state=0)

rf_bagging_model.fit(x_train , y_train)
rf_bagging_train_score = rf_bagging_model.score(x_train , y_train)

print("RandomForest Bagging Classifier Training Score: {:.3F}".format(rf_bagging_train_score))


# BaggingClassifier evaluated using shuffle-split cross-validation 
rf_bagging_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.2, n_splits=5, random_state=0)
rf_baggign_val_scores = cross_val_score(rf_bagging_model, x_train , y_train, cv=rf_bagging_shuffle_split)
print("RandomForest Bagging Classifier Cross validation Score: {:.3F}".format(np.mean(rf_baggign_val_scores)))

#### 3.3.3.1- pasting

In [ ]:
rf_pasting_model = BaggingClassifier(base_estimator=RandomForestClassifier(max_depth=8,
                                                                             max_leaf_nodes=10,
                                                                             n_estimators=100,
                                                                             n_jobs=-1,
                                                                             random_state=0),
                                     n_estimators=100,
                                     bootstrap=False, #The difference between baggong and pasting
                                     max_samples=100,
                                     n_jobs=-1,
                                     random_state=0)

rf_pasting_model.fit(x_train , y_train)
rf_pasting_train_score = rf_pasting_model.score(x_train , y_train)

print("RandomForest pasting Classifier Training Score: {:.3F}".format(rf_pasting_train_score))


# BaggingClassifier evaluated using shuffle-split cross-validation 
rf_pasting_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.2, n_splits=5, random_state=0)
rf_pasting_val_scores = cross_val_score(rf_pasting_model, x_train , y_train, cv=rf_pasting_shuffle_split)
print("RandomForest pasting Classifier Cross validation Score: {:.3F}".format(np.mean(rf_pasting_val_scores)))

### 3.3.4- Boosting

#### 3.3.4.1- AdaBoost

In [ ]:
rf_adaboost_model = AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=5,
                                                                             max_leaf_nodes=10,
                                                                             n_estimators=100,
                                                                             n_jobs=-1,
                                                                             random_state=0),
                                       
                                       n_estimators=100,
                                       learning_rate=0.1,
                                       algorithm='SAMME.R',
                                       random_state=0)

rf_adaboost_model.fit(x_train , y_train)
rf_adaboost_train_score = rf_adaboost_model.score(x_train , y_train)

print("RandomForestClassifier AdaBoost Classifier Training Score: {:.3F}".format(rf_adaboost_train_score))


# AdaBoostClassifier evaluated using shuffle-split cross-validation 
rf_adaboost_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.2, n_splits=5, random_state=0)
rf_adaboost_val_scores = cross_val_score(rf_adaboost_model, x_train , y_train, cv=rf_adaboost_shuffle_split)
print("RandomForestClassifier AdaBoost Classifier Cross validation Score: {:.3F}".format(np.mean(rf_adaboost_val_scores)))

#### 3.3.4.2- GradientBoostingClassifier

In [ ]:
g_boosting_model = GradientBoostingClassifier(n_estimators=500,
                                             learning_rate=0.01,
                                             random_state=0,
                                             max_depth=5)

g_boosting_model.fit(x_train , y_train)
g_boosting_train_score = g_boosting_model.score(x_train , y_train)

print("Gradient Boosting Classifier Training Score: {:.3F}".format(g_boosting_train_score))


# GradientBoostingClassifier evaluated using shuffle-split cross-validation 
g_boosting_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.2, n_splits=5, random_state=0)
g_boosting_val_scores = cross_val_score(g_boosting_model, x_train , y_train, cv=g_boosting_shuffle_split)
print("Gradient Boosting Classifier Cross validation Score: {:.3F}".format(np.mean(g_boosting_val_scores)))

# overfitting

# 4- Deep learning  classification model.

I will Neural network that consists of 3 layers with different number of neurons, the atication fumction will be relu 
and for the final layer will be sigmoid because we have 2 classes, BatchNormalization layers and Dropout layers after 
the Dense layers. 

In [ ]:
inputs = tf.keras.layers.Input(shape=(30,))

x = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal',
                               kernel_regularizer=tf.keras.regularizers.L2())(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Dense(128, activation="relu", kernel_initializer='he_normal',
                               kernel_regularizer=tf.keras.regularizers.L2())(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)


output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

nn_model = tf.keras.Model(inputs, output)

In [ ]:
nn_model.compile(optimizer='rmsprop',
                 loss='categorical_crossentropy',
                 metrics=["acc"])

In [ ]:
tf.keras.utils.plot_model(nn_model, show_shapes=True, rankdir="LR")

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='acc',
                                                  patience=10,
                                                  restore_best_weights=True)

In [ ]:
history = nn_model.fit(x_train, y_train, epochs=400,
                       batch_size=64,
                       callbacks=[early_stopping])

In [ ]:
nn_train_acc = nn_model.evaluate(x_train, y_train)[1]
nn_test_acc = nn_model.evaluate(x_test, y_test)[1]

# 5- Final Analysis and comparison.

## 5.1- comparison

In [ ]:
train_scores = [lr_train_score, dt_train_score,
                rf_train_score, hard_voting_train_score,
                soft_voting_train_score, 
                rf_bagging_train_score, rf_pasting_train_score,
                rf_adaboost_train_score, g_boosting_train_score]


cross_val_scores = [lr_val_scores, dt_val_scores,
                    rf_val_scores, hard_voting_val_scores,
                    soft_voting_val_scores, rf_baggign_val_scores,
                    rf_pasting_val_scores, rf_adaboost_val_scores,
                    g_boosting_val_scores]

In [ ]:
mean_cross_val_scores = []

for i in cross_val_scores: 
    mean_cross_val_scores.append(np.mean(i))


In [ ]:
models = [lr_model, dt_model, rf_model,
          hard_voting_model, soft_voting_model,
          rf_bagging_model, rf_pasting_model,
          rf_adaboost_model, g_boosting_model]


test_scores = []

for model in models:
    test_score = model.score(x_test, y_test)
    test_scores.append(test_score)

In [ ]:
models = pd.DataFrame({
    'Model': [ 'Logistic Regression','Decision Tree Classifier',
               'Random Forest',
               'Hard Voting Classifiers', 'Soft Voting Classifiers',
               'Bagging Classifier using RandomForestClassifier',
               'Pasting Classifier using RandomForestClassifier',
               'AdaBoost Classifier using RandomForestClassifier',
               'Gradient Boosting Classifier'],
    
    
    'Training Accuracy': train_scores,
    
    
    'Cross Validation Accuracy': mean_cross_val_scores,

    'Testset Accuracy': test_scores})



In [ ]:
models.loc[len(models.index)] = ['Neural Network', nn_train_acc, '-', nn_test_acc]

models

As we can see from  the table above the models achieve good testing result. The highest testset Accuracy 
was for the Logistic Regression and Hard Voting Classifiers I think that because the data is very simple and
small to use a neural network. the second best scores were for Random Forest, Bagging, Pasting, and Neural Network. 

In the next step I will choose one or two of these models to further investigation and analysis. 

I will choose the Logistic Regression and the Random Forest	

## 5.2- (Trying) to train the model after applying PCA 

For trying and to get more insights I will reduce the feature space for the data and train the model again to see
if that will have any impact on the accuracy. 

In [ ]:
pca = PCA(n_components=10)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
pca_lr_model = LogisticRegression(max_iter=1000, random_state=0)
pca_lr_model.fit(x_train_pca , y_train)
Pca_lr_train_score = pca_lr_model.score(x_train_pca , y_train)

print("Logistic Regression Training Score: {:.3F}".format(Pca_lr_train_score))


# LogisticRegression evaluated using shuffle-split cross-validation 
pca_lr_shuffle_split = StratifiedShuffleSplit(train_size=0.8, test_size=0.15, n_splits=5, random_state=0)
pca_lr_val_scores = cross_val_score(pca_lr_model, x_train_pca , y_train, cv=pca_lr_shuffle_split)
print("Logistic Regression Cross validation Score: {:.3F}".format(np.mean(pca_lr_val_scores)))

In [ ]:
pca_test_score = pca_lr_model.score(x_test_pca, y_test)
pca_test_score

After reducing the dimensionality of dataset the Cross validation Score was increazed by about 2% but
the testset score was decreased by about 2% 

## 5.3- Further analysis

In [ ]:
cf_matrix = confusion_matrix(y_test, lr_model.predict(x_test))
sns.heatmap(cf_matrix, annot=True)

In [ ]:
print("The number of True Positive is: {}".format(cf_matrix[0, 0]))
print("The number of True Negative is: {}".format(cf_matrix[1, 1]))
print("The number of False Positive is: {}".format(cf_matrix[0, 1]))
print("The number of False Negative is: {}".format(cf_matrix[1, 0]))

In [ ]:
Logistic_Regression_Precision_score = precision_score(y_test,
                                                      lr_model.predict(x_test))

print(" Logistic Regression Precision score: {}".format(Logistic_Regression_Precision_score))

In [ ]:
Logistic_Regression_recall_score = recall_score(y_test,
                                                lr_model.predict(x_test))

print(" Logistic Regression Recall score: {}".format(Logistic_Regression_recall_score))

In [ ]:
Logistic_Regression_f1_score = f1_score(y_test,
                                        lr_model.predict(x_test))

print(" Logistic Regression F1 score: {}".format(Logistic_Regression_f1_score))

In [ ]:
Logistic_Regression_Average_Precision_score = average_precision_score(y_test,
                                                                      lr_model.predict_proba(x_test)[:,1])

print("Logistic Regression Average Precision score: {}".format(Logistic_Regression_Average_Precision_score))

In [ ]:
Logistic_Regression_ROC_AUC_score = roc_auc_score(y_test,
                                                  lr_model.predict_proba(x_test)[:,1])

print("Logistic Regression ROC AUC Score: {}".format(Logistic_Regression_ROC_AUC_score))

# 6- Conclusion

Heart Disease UCI dataset was used in this notebook, first we performed an exploratory data analysis to analyze
and understand the dataset we have. T have started with a visualization of the missing data, then I start to analyze the 
features one by one. 


The mean and the standard deviation of the ages for the people have and dont have the disease  are
so close for both, and from the means and the histogram we can say that the younger people are more susceptible to disease. 
The number of males in this dataset is more than twice the number of females, and the number of males that have the
disease is higher than females that have the disease the number of males that do not have the disease is higher than
females that have the disease. The most frequent pain type between people have the disease is type 2.


the people havethe disease tend to have a lower resting blood pressure and tend to have a lower serum cholesterol in mg/dl.
the number of people that have the disease and fasting blood sugar > 120 mg/dl (class 1) is almost equal to the number of
people that do not have the disease and fasting blood sugar < 120 mg/dl (class 0).


Also for the people have the disease the most resting electrocardiographic results type was type 1, and they have number of
major vessels 0 (class 0), also they have the (class 2) of the slope of the peak exercise ST segment
while most of the people dont have the disease have the (class 1)


The correlation between the valriables and the target was studied using Spearman’s ρ and most of the variables was related the 
target, some them are positively related and other are negatively.


The data was cleaned from the outliers then  prepared for the macine learning models, the age feature was binned, other numerical features passed through  a pipeline that extract new features then select the best of them using the RFE and 
finnaly normalized. For the categorical data I used the one_hot encoder to encode them. 


Several Machine learning model were tested the highest testset Accuracy was for the Logistic Regression
and Hard Voting Classifiers I think that because the data is very simple and small to use a neural network.
the second best scores were for Random Forest, Bagging, Pasting, and Neural Network.


Finally the Logistic regreesion model was chosen for further analysis and the following metrics were computed: precision_score,
recall_score, f1_score, average_precsions_score, roc_auc_score and all of them shown a very good results, all of them
were higher than 95%. 

# 7- Thank you 

**Thank you for reading, I hope you enjoyed and benefited from it.** <br>
 
**If you have any questions or notes please leave it in the commont section.** <br>

**If you like this notebook please press upvote and thanks again.** <br>